In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
os.getcwd()

In [ ]:
conn = sqlite3.connect(r'C:\Users\joshd\OneDrive\Desktop\culminating-project-group-2-1\Database\team2_DB.db')

query = """SELECT Country_Name, Country_Code, Time, Time_Code, Broad_Money_Perc, 
        Broad_Money_Growth, Broad_Money_Reserves, Broad_Money_Total, Exchange_Rate  
FROM Statistical_Indicators
WHERE (SELECT count("Country_Code") FROM Statistical_Indicators GROUP BY Country_Code) = 3
AND Broad_Money_Perc
AND Broad_Money_Growth
AND Broad_Money_Reserves
AND Broad_Money_Total
AND Exchange_Rate <> '..' """

df = pd.read_sql_query(query, conn)

In [ ]:
# create list of unique countries

unique_countries = df['Country_Name'].unique().tolist()
count_countries = len(unique_countries)
print(unique_countries)
print(count_countries)

In [ ]:
# Conduct PCA on Broad Money columns

pca_columns = ['Broad_Money_Perc', 'Broad_Money_Growth', 'Broad_Money_Reserves', 'Broad_Money_Total']

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[pca_columns])

pca = PCA(n_components=2) 
pca_result = pca.fit_transform(scaled_data)

pca_df = pd.DataFrame(data=pca_result, columns=['BM_PC1', 'BM_PC2'])
print(pca_df.head())

In [ ]:
# Standardize Exchange Rate to fall between -1 and 1 with 100 as the 0
df['Exchange_Rate_Standardized'] = (df['Exchange_Rate'] - 100) / 100
print(df[['Exchange_Rate', 'Exchange_Rate_Standardized']].head(20))

In [ ]:
df = pd.concat([df, pca_df], axis=1)
print(df.head())

In [ ]:
df.info()

In [28]:
df.describe()

,Time,Exchange_Rate,Exchange_Rate_Standardized,BM_PC1,BM_PC2
count,342.000000,342.000000,342.000000,3.420000e+02,3.420000e+02
mean,2021.944444,703.068139,6.030681,-1.558208e-17,-2.597013e-17
std,0.813999,2608.903047,26.089030,1.142007e+00,1.094269e+00
min,2021.000000,0.301643,-0.996984,-1.060330e+00,-5.919570e+00
25%,2021.000000,3.223718,-0.967763,-4.507902e-01,-1.743834e-01
50%,2022.000000,18.673717,-0.813263,-2.931419e-01,1.745878e-01
75%,2023.000000,139.370526,0.393705,-1.312093e-02,4.821150e-01
max,2023.000000,23271.212500,231.712125,1.387217e+01,7.101879e+00
